#imports



In [3]:
from google.colab import drive
drive.mount('/content/drive')
!pip install ipynb
import sys
sys.path.insert(0,'/content/drive/MyDrive/CNN-deterministic-decoupling')
path = '/content/drive/MyDrive/CNN-deterministic-decoupling'
import torch
import math
from ipynb.fs.defs.imageFunctions import prepare_image,show_image,save_image
from ipynb.fs.defs.Runge_Kutta import RungeKutta4
from ipynb.fs.defs.DISTS_mod import DISTS
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Hacia atras

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
target = prepare_image(Image.open(path+"/imagenes/texture6.jpg").convert("RGB"),size=128)
target=target.to(device)
target.requires_grad_(False)

show_image(target)


origin=prepare_image(Image.open(path+"/imagenes/ruido3.png").convert("RGB"),size=128)
#origin=torch.normal(mean=0.45, std=0.22,size=target.size())
#transform = transforms.Resize((256, 256))
#origin = transform(origin)

show_image(origin)

origin=origin.to(device)
origin.requires_grad_(True)



x0=origin.detach().clone()
x0=x0.to(device)


model_d = DISTS().to(device)
model_d.set_target(target)

epoch=0
step_size = 0.1

inicial_Loss=model_d.forward(origin).item()
LossList=[inicial_Loss]

stopping_criterion=0.0005

while LossList[-1] > stopping_criterion:
    RungeKutta4(origin,model_d.forward,step_size,LossList)
    epoch+=1
    if epoch%2000==0:
      print("iteracion "+str(epoch))
      print("loss "+str(LossList[-1]))
      show_image(origin)
      with open(path+'/Datos/Sintesis_6_RK4_datos.txt', 'w') as f:
        f.write(f'Total iterations: {epoch}\n')
        f.write('\n')
        f.write(f'Loss values: {LossList}\n')
      save_image(origin,path+'/Datos/Sintesis_6_RK4.png')

print("iteracion "+str(epoch))
print("loss "+str(LossList[-1]))
show_image(origin)
with open(path+'/Datos/Sintesis_6_RK4_datos.txt', 'w') as f:
  f.write(f'Total iterations: {epoch}\n')
  f.write('\n')
  f.write(f'Loss values: {LossList}\n')
save_image(origin,path+'/Datos/Sintesis_6_RK4.png')


In [ ]:
plt.plot(EpochList,LossList)
plt.grid(True)

In [ ]:
plt.plot(LossList,ErrorList)
plt.grid(True)